# AE2 - The Colour Language Game

#### Importing Libaries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections as col 
import re
import nltk
from nltk import word_tokenize, pos_tag
import random
import math
from PIL import Image
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
import networkx as nx
import collections as col
from collections import Counter
import webcolors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import warnings
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

#### Cleaning color database

In [ ]:
colorsDF = pd.read_csv('colour_naming_data.csv', encoding='ISO-8859-1')

# Renaming unlabled columns
newColumnNames = {
    'Unnamed: 1': 'Color Name',
    'Unnamed: 2': 'Red',
    'Unnamed: 3': 'Green',
    'Unnamed: 4':'Blue'
}
colorsDF = colorsDF.rename(columns=newColumnNames)

# Dropping reference lines
colorsDF = colorsDF.drop(colorsDF.index[:4], axis=0)

# Removing Sample ID Column
colorsDF = colorsDF.drop(colorsDF.columns[0], axis=1)

# Removing whitespace, making lowercase, and removing special characters
colorsDF = colorsDF.applymap(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x.strip().lower()) if isinstance(x, str) else x)

# Change RGB values to int
columnsToConvert = ['Red', 'Green', 'Blue']
colorsDF[columnsToConvert] = colorsDF[columnsToConvert].astype(int)

# Concentrate on main hue component by removing adjectives
def removeColorAdjectives(text):
    colorAdjectives = ['pale', 'vivid', 'soft', 'muted', 'very', 'saturated','grass', 'deep', 'appale', 'dull']  
    pattern = r'\b(?:' + '|'.join(colorAdjectives) + r')\b\s*'
    cleanedText = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return cleanedText

colorsDF['Color Name'] = colorsDF['Color Name'].apply(removeColorAdjectives)

#### Helper Methods

In [ ]:
def L2Distance(color1, color2):
    """
    Euclidean distance between colors
    """
    assert len(color1) == len(color2) == 3, "Color tuples misarranged"
    
    return math.sqrt((color1[0]-color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2])**2)


def findCentroid(color):
    """
    Returns averaged RGB tuple of given color
    """
    # Take only the specific color subset of dataframe
    colorValues = colorsDF[colorsDF['Color Name'] == color]
    # find the centroid (average RGB values of given color)
    averageValues = tuple(colorValues[['Red', 'Green', 'Blue']].mean())
    return averageValues


def isColor(string):
    """
    Returns true if a given color is in the webcolors database, false if not
    """
    try:
        webcolors.name_to_rgb(string)
        return True
    except ValueError:
        return False
    
def threeDToTwoD(colorCoord):
    """
    Converts a 3 dimensional list or tuple into 2 dimensions
    """
    return (colorCoord[0]/colorCoord[2], colorCoord[1]/colorCoord[2])

def isCorrectColor(color, colorRGB, threshold):
    """
    Returns true if the distance between a color and its centroid is below a certain threshold, false if not
    """
    dist = L2Distance(colorRGB, findCentroid(color))
    return dist <= threshold

def rgbListToColorList(model, rgbVals):
    """
    Creates a list of guesses with a KNN model converting rgb values to color names
    """
    guesses = []
    for color in rgbVals:
        RGBValuesScaled = scaler.transform(np.array(color).reshape(1,-1)) 
        predictedColor = model.predict(RGBValuesScaled)
        guesses.append(predictedColor)
    guesses = [guess[0] for guess in guesses]
    return guesses

#### Alice Picking Colors

In [ ]:
def generateRandomColor():
    """
    Returns a tuple of 3 values between 0-255
    """
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
aliceTestColors = [generateRandomColor() for _ in range(10)]

print("Alice has chosen the following colors:")
for i, color in enumerate(aliceTestColors):
    print(f"Color {i+1}: {color}")

#### Bob Answers

In [ ]:
warnings.filterwarnings('ignore') 

# Pull out rgb data and color name answers
X = colorsDF[['Red', 'Green', 'Blue']]  # Features: RGB values
y = colorsDF['Color Name']  # Target labels: Color names

# Scale features 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


def totalDistance(k):
    """
    Returns the total euclidean distance between centriods and 100 randomly generating colors
    """
    testColors = [generateRandomColor() for _ in range(100)]
    
    # train the model based on k parameter
    knnModel = KNeighborsClassifier(n_neighbors=k)
    knnModel.fit(X_scaled, y)
    
    guesses = rgbListToColorList(knnModel, testColors)
    
    totalDistance = 0.0
    centriods = [findCentroid(color) for color in guesses]
    for i, col in enumerate(guesses):
        if any(isinstance(x, float) and math.isnan(x) for x in centriods[i]): # NAN check
            continue
        else:
            totalDistance += L2Distance(testColors[i], centriods[i])
    
    return totalDistance


def bestK():
    """
    Returns best value for K based on the the one which gives the 
    lowest total distance between colors and their centriods.
    Secondly, returns the list of summed distances computed for each K
    """
    bestDistance = float('inf')
    bestK = -1
    distancesList = []
    for i in range(1, 13):
        distancesList.append(totalDistance(i))
    bestK = distancesList.index(min(distancesList))
    return bestK+1, distancesList
    

# Train classifier with best K
bestKValue = bestK()[0]  # Use best value of K (tends to be 3)
knnModel = KNeighborsClassifier(n_neighbors=bestKValue)
knnModel.fit(X_scaled, y)

# Compute KNN (Bob's) guesses
bobGuesses = []

for color in aliceTestColors:
    new_rgb_values_scaled = scaler.transform(np.array(color).reshape(1,-1))  # Scale the new data
    predicted_color = knnModel.predict(new_rgb_values_scaled)
    bobGuesses.append(predicted_color)
    
print("Hi, I'm Bob. Here's what I think those colors are: ")
for i, color in enumerate(bobGuesses):
    print(f"Color {i+1} is {color[0]}")
    
def displayColors(rgbValues, colorNames, boxWidth = 2):
    """
    Prints squares of colors and their name above them
    """
    
    numColors = len(rgbValues)
    
    fig, ax = plt.subplots(1, 1, figsize=(numColors * boxWidth, 1))
    ax.set_axis_off()
    
    for i in range(numColors):
        color = tuple(val / 255 for val in rgbValues[i])  # Normalize RGB values
        name = colorNames[i][0].title()
        
        ax.text(i + 0.5, 1, name, ha='center')
        ax.add_patch(plt.Rectangle((i, 0), 1, 1, color=color))
    
    ax.set_xlim(0, numColors)
    ax.set_ylim(0, 1)
    plt.show()

displayColors(aliceTestColors, bobGuesses)
print('\n')

def colorScatterPlot(rgbValues, colorNames):
    """
    Prints a 3D scatter plot from rgb values labeled with color names
    """
    # Convert RGB values to floats between 0 and 1
    rgbFloat = np.array(rgbValues) / 255.0

    # Create scatter plot
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(rgbFloat[:, 0], rgbFloat[:, 1], rgbFloat[:, 2], c=rgbFloat, s=500)

    # Add labels to each point
    for i, txt in enumerate(colorNames):
        ax.text(rgbFloat[i, 0], rgbFloat[i, 1], rgbFloat[i, 2], txt, color='black', fontsize=12)

    ax.set_xlabel('Red')
    ax.set_ylabel('Green')
    ax.set_zlabel('Blue')
    ax.set_title('RGB Scatter Plot with Color Labels)')

    # Make the plot interactive
    plt.show()
    
colorScatterPlot(aliceTestColors, bobGuesses)

#### Evaluating Bob's performance

In [ ]:
# Evaluate performance

def accuracy(model, testColors, threshold = 80, nFolds = 10):
    """
    Returns mean accuracy for model
    """
    
    # Generate the testing colors
    testColors = []
    for i in range(nFolds):
        testColors.extend([generateRandomColor() for _ in range(20)])
    
    # Generate color guesses
    guesses = rgbListToColorList(knnModel, testColors)
    
    # Calculate accuracy
    totalGuesses = len(testColors)
    correctGuesses = 0

    for rgbVal, color in zip(testColors, guesses):
        if isCorrectColor(color, rgbVal, threshold):
            correctGuesses +=1
    
    return (correctGuesses/totalGuesses) * 100

def classificationMetrics(predictions, groundTruth, color):
    """
    Calculates true positives and negatives and false positives and negatives
    """
    TP = sum(1 for pred, gt in zip(predictions, groundTruth) if pred == color and gt == color)
    TN = sum(1 for pred, gt in zip(predictions, groundTruth) if pred != color and gt != color)
    FP = sum(1 for pred, gt in zip(predictions, groundTruth) if pred == color and gt != color)
    FN = sum(1 for pred, gt in zip(predictions, groundTruth) if pred != color and gt == color)
    
    return TP, TN, FP, FN

def precisionCalc(rgbColorList, colorNameList, groundTruths):
    """
    Returns average precision of color set 
    """
    
    total = 0
    uniqueColors = set(colorNameList)
    for color in uniqueColors:
        TP, TN, FP, FN = classificationMetrics(colorNameList, groundTruths, color)
        total += TP/(TP + FP)
    
    return total/len(uniqueColors)

def recallCalc(rgbColorList, colorNameList, groundTruths):
    """
    Returns the recall score
    """
    total = 0
    
    uniqueColors = set(colorNameList)
    for color in uniqueColors:
        TP, TN, FP, FN = classificationMetrics(colorNameList, groundTruths, color)
        if (TP + FN) > 0:
            total += TP / (TP + FN)
        else:
            continue
            
    return total/len(uniqueColors)

def f1Calc(precision, recall):
    """
    Returns f1 score
    """
    return 2 * (precision * recall) / (precision + recall)

# Start printing evals

# Getting a consistent testset with rgb values and color names
testColors = [generateRandomColor() for _ in range(100)]

guesses = rgbListToColorList(knnModel, testColors)

# create the list of ground truths to determine accuracy 
groundTruths = [colorName if isCorrectColor(colorName, colorRGB, 70) else 'xxx' 
                for colorRGB, colorName in zip(testColors, guesses)]

print(f"Bob's mean accuracy was: {accuracy(knnModel, testColors)}%")
precision = precisionCalc(testColors, guesses, groundTruths)
print(f"Bob's precision was: {precision*100:.2f}%")
recall = recallCalc(testColors, guesses, groundTruths)
print(f"Bob's recall was: {recall*100:.2f}%")
print(f"Bob's f1-score was: {f1Calc(precision, recall)*100:.2f}%")

# Visualizing

# Values for K
x = range(1, 13)

# list of all summed euclidian distances for each k value 1-12. It is metric 1
distances = bestK()[1]

# list of accuracy scores for each k, it is metric 2
acc = []

# list of f1 scores for each k, it is metric 3
f1 = []

# Calculates metrics for K 1-12
for i in range(1,13):
    variousKModel = KNeighborsClassifier(n_neighbors=i)
    variousKModel.fit(X_scaled, y)
    newGuesses = rgbListToColorList(variousKModel, testColors)
    prec = precisionCalc(testColors, newGuesses, groundTruths)
    recall = recallCalc(testColors, newGuesses, groundTruths)
    acc.append(accuracy(variousKModel, testColors))
    f1.append(f1Calc(prec, recall)*100)


fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot the first line with thousands on the left y-axis
sns.lineplot(x=x, y=distances, ax=ax1, label='Total Distance from Centroids', color='blue')
ax1.set_ylabel('Y-axis (Line 1)', color='blue')

# Create a second y-axis sharing the same x-axis
ax2 = ax1.twinx()

# Plot the other lines with tens on the right y-axis
sns.lineplot(x=x, y=acc, ax=ax2, label='Accuracy', color='green')
sns.lineplot(x=x, y=f1, ax=ax2, label='F1-Score', color='red')
ax2.set_ylabel('Y-axis (Lines 2 and 3)', color='black')

plt.title('K Values vs Various Metrics')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()

In [ ]:
class Graph:
    """Graph class, for an undirected, unweighted graph with an adjacency list representation
        For an edge A-B, it appears in the adjaency list as A-B and B-A
    """

    def __init__(self, V = [], E = []):
        """ Create a new graph from a list of verticies V and edges E.
        By default, the graph is undirected """
        self.G = {}

        for v in V:
            self.add_vertex(v)
        for u, v in E:
            self.add_edge(u, v)
            
    def add_vertex(self, v):
        if v not in self.G:
            self.G[v] = set()
        
    def add_edge(self, u, v):
        # add vertices in case they don't already exist
        self.add_vertex(u)
        self.add_vertex(v)

        # add undirected edge (u,v)
        self.G[u].add(v)
        self.G[v].add(u)
            
        
    def __getitem__(self, v):
        """ Return all vertices adjacent to v (overriding index operator!)"""
        return self.G.get(v, set())

    def bfs_spanning_tree(self, start):
        """ Find the breadth-first search spanning tree
        This algorithm uses the double-ended queue collection class """
        span = DirectedGraph(V=[start])
        Q = col.deque() # Use the built-in double-ended queue
        Q.appendleft(start)
        visited = {start}
        
        while len(Q)>0: # while Q is not empty
            current = Q.pop()
            adj = self.G[current]  # find vertices adjacent to current node
            for v in adj:          # for each vertice, if they are not yet visited, add a spanning edge
                if v not in visited:
                    Q.appendleft(v)
                    visited.add(v)
                    span.add_edge(current,v)
        
        return span
    
    def shortest_path(self, start, end):
        """ Find the shortest path from start to end. This implementation is not the most efficient.
        It first finds the BFS spanning tree, reverses all of the edges in the BFS tree then
        walks from the end to the start to determine the shortest path in the original graph. """
        bfs = self.bfs_spanning_tree(start) # creates a digraph
        rev = bfs.reverse()
        path = rev.walk(end, start)
        return path

    def __repr__(self):
        graph_str = ''
        for v in self.G:
            graph_str += '['+v+'] => ' + str(self[v]) + '\n'
        return graph_str

    def is_adjacent(self,u,v):
        """ Test if u and v are adjacent """
        return v in self[u] and u in self[v]
    
    def get_vertices(self):
        """ Get a list of all vertices in the graph """
        return list(self.G.keys())

    def get_edges(self):
        """ Return a list of edges in the graph.  Each edge is a tuple (u,v) """
        edges = []
        for u in self.G:
            for v in self.G[u]:
                edges.append((u,v))
        return edges

    def num_vertices(self):
        """ Return the number of vertices in the graph """
        return len(self.G)

    def num_edges(self):
        """ Return the number of edges in the undirected graph """

        total = 0
        for v in self.G:
            total += self.deg(v)

        return total // 2

    def deg(self,v):
        """ What is the degree of vertext v?  i.e., how many 
        other vertices are adjacent """
        return len(self[v])

    def degree_distribution(self):
        """ Compute the degree distribution of the graph 
        Return a dictionary (key=degree, value=# vertices of that degree. """
        dd = {}
        for v in self.G:
            degree = self.deg(v)
            dd[degree] = dd.get(degree,0) + 1
        return dd

    def toDF(self,columns=['u', 'v']):
        """ Convert the graph to a pandas dataframe representation """
        df = pd.DataFrame(columns = columns)
        for (u,v) in self.get_edges():
            df = df.append({columns[0]:u, columns[1]:v}, ignore_index=True)
        return df

    def fromDF(self, df, columns=['u', 'v']):
        """ Convert from a pandas dataframe with two columns 
        identifying the vertices.  Each row is an edge. """
        edges = list(zip(df[columns[0]], df[columns[1]]))
        for u,v in edges:
            self.add_edge(u,v)

    def visualize(self, fig = 1, directed=False):
        """ Render the graph using networkx library """
        
        nx_graph = nx.Graph()
        
        for vertex1 in self.get_vertices():
            for vertex2 in self.G[vertex1]:
                dist = L2Distance(findCentroid(vertex1), findCentroid(vertex2))
                nx_graph.add_edge(vertex1, vertex2, weight=(dist//65))

        positions3d = {}
        for color in self.get_vertices():
            positions3d[color] = findCentroid(color)
        
        positions2d = {}
        for key, value in positions3d.items():
            positions2d[key] = threeDToTwoD(value)
            
        colorMap = {}
        for color in self.get_vertices():
            colorMap[color] = color
        
        plt.figure(figsize=(22, 22))
        nx.draw(nx_graph, pos=positions2d, with_labels=True, node_color=[colorMap[node] for node in nx_graph.nodes()], 
                node_size=1200, font_size=8, width=[d['weight'] for u,v,d in nx_graph.edges(data=True)])
        plt.show()
       

    def subgraph(self,V):
        """ Find the subgraph consisting only of the specified 
        vertices and the edges that occur between those vertices """
        sub = Graph()
        
        # Add vertices
        for u in V:
            sub.add_vertex(u)
            
        # Add edges
        for u in V:
            for v in V:
                if v in self[u]:
                    sub.add_edge(u,v)
        return sub

#### Alice Responce

In [ ]:
# Plotting Distance between guesses and centroids for each color Bob guessed
bobCentroids = [findCentroid(color[0]) for color in bobGuesses]
distanceBetweenGuessesAndCentroids = [L2Distance(color1, color2) for color1, color2 
                                      in zip(bobCentroids, aliceTestColors)]

colorPositions = np.arange(len(bobGuesses))

for i, (color, number) in enumerate(zip(bobGuesses, distanceBetweenGuessesAndCentroids)):
    plt.bar(colorPositions[i], number)

plt.xticks(colorPositions, bobGuesses, rotation = 90)

plt.xlabel('Color')
plt.ylabel("Distance")
plt.title("Euclidean Distance between Centroid of Bob's Guesses and Alice's Values (in order of guesses)")
plt.show()


# Graph of all (relevant) colors and distance between centroids as edges

colorGraph = Graph()

# Get all unique colors from the dataframe by just taking the last word in name (ie. purpulish blue becomes blue)
uniqueColors = list(colorsDF['Color Name'].unique())
uniqueColors = [color.split()[len(color.split())-1] for color in uniqueColors if color != '']
uniqueColors = list(dict.fromkeys(uniqueColors))

# Series with color names and their frequencies in the dataset
colorFreq = colorsDF['Color Name'].value_counts()

# Sorting out all colors with a frequency below 5 (irrelvant colors)
illegalColors = []
for c, num in colorFreq.items():
    if num < 5:
        illegalColors.append(c)
    
    
uniqueColors = [color for color in uniqueColors if isColor(color)]
for color in illegalColors:
    if color in uniqueColors:
        uniqueColors.remove(color)
        

# add all remaining colors to graph
for color in uniqueColors:
    colorGraph.add_vertex(color)
    for color2 in uniqueColors:
        if color2 != color:
            colorGraph.add_edge(color, color2)

colorGraph.visualize()



## Going Above and Beyond

In [ ]:
def mostFreqColors():
    
    """
    Maps the most frequently appearing colors
    """

    # Get only relevant colors (take last word of color name then filter out frequencies under 50)
    colorsDF['Color Name'] = colorsDF['Color Name'].apply(lambda x: x.split()[-1] if x else '')
    colorFreq = colorsDF['Color Name'].value_counts()
    colorFreqFiltered = colorFreq[colorFreq > 50]

    # Plotting
    plt.figure(figsize=(10, 6))
    colorFreqFiltered.plot(kind='bar')
    plt.title('Frequency of Colors')
    plt.xlabel('Color')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
mostFreqColors()
print("Demonstrates a bias toward popular colors")


def plotMisclassColors():
    """
    Maps the most frequently misclassified colors
    """
    misclassColors = {}

    testColors = []
    testColors.extend([generateRandomColor() for _ in range(250)])

    guesses = rgbListToColorList(knnModel, testColors)

    totalGuesses = len(testColors)
    correctGuesses = 0

    # counting the color appearance count and misclassifcation count in value
    for rgbVal, color in zip(testColors, guesses):
        if isCorrectColor(color, rgbVal, 120):
            continue
        else:
            if color in misclassColors:
                misclassColors[color] += 1
            else:
                misclassColors[color] = 1

    misclassColorsSorted = dict(sorted(misclassColors.items(), key=lambda item: item[1], reverse = True))
    misclassColorsFiltered = {key: value for key, value in misclassColorsSorted.items() if value > 1}

    plt.bar(list(misclassColorsFiltered.keys()), list(misclassColorsFiltered.values()))
    plt.xticks(rotation = 90)

    # Adding labels
    plt.xlabel('Colors')
    plt.ylabel('Number of Misclassifcations')
    plt.title('Most Often Misclassified Colors')

    # Display the chart
    plt.show()
    
plotMisclassColors()


## Relection

This project was beneficial for me in understanding the critical concepts of supervised learning. By using KNN, I understood how data is processed, how models are deployed, how to tune the model's hyperparameters, and how to use metrics to evaluate performance. Preprocessing the data took some time as I had to remove overly specific color names by filtering out adjectives such as 'saturated', 'soft', and 'muted'. Deploying the model was also a valuable task as I had never used Sci-Kit Learn before and found how easy it was to build the model. The hyperparamiterzation of K was somewhat challenging as I had to use an n-fold cross-validation metric to find which value of K minimized the total Euclidian distance between RGB colors and their centroids. That value of K tended to be relatively low (most often 3). Finally, the metrics I mapped helped confirm my value of K as the F1 score and accuracy score often followed the total distance measure for each value of K.

My biggest challenge was creating my own distance metrics. The Sci-Kit Learn library is not practical in testing this model because if we generate random values to test the model with, we have no answers (or ground truths) to compare them to. I found the dataset had too many unique colors and a significantly unbalanced distribution of colors (ie far more greens and blues than reds and oragnes). I tried to fix this issue using Sci-Kit's startified K fold validation (which takes test sets that uphold percecntage distributions of each color) but my accuracy remained too low (around 0.38 or 38%). For the reasons listed above I belive Sci Kit is fundamentally somewhat flawed for this problem.

Therefore, I generated my own method of creating ground truth by making a function that returns True if the distance between a color and its centroid was under a certain threshold (usually 80). I found the threshold of 80 to be effective through trial and error. I went on an RGB visualization site, plugged in various colors, and calculated their Euclidian distances. In my opinion, any 2 colors with a distance of 80 or less could undeniably be categorized as the same color. With a distance of 90 or greater, that assertion can be questioned in some instances. 

Therefore, I generated 300 random values. Determined their groundtruths. Then, I used that information to calculate a more accurate accuracy, F1, recall, and precision score than Sci-Kit Learn, which has no real knowledge of the circumstances. 

Another challenge I faced was determining how many colors to filter out from Alice's graph. It was a tradeoff between readibility of the vertexes and their labels and the completness of the graph.


For future iterations of this project, I would recommend having students manually implement KNN. I did it to begin before the lecture about Sci-Kit Learn, and it worked nearly as well. I deleted it because the assignment was to use Python libraries, but it is a relatively easy algorithm to implement, and I feel it would give students a better understanding of what is going on behind the scenes. I also think it would give a more satisfying result to students when it works as they would feel they have more ownership of it rather than typing 3 lines calling Python libraries. Furthermore, it would allow students to hyperparameterize another variable: the distance function as well as K. This was the most interesting part of the project for me, so having another parameter to optimize would be fun. Lastly, it would lead to further interesting visualizations based on Euclidean, Hamming, Manhatten, etc.; distance functions students would implement. They could map performance based on each function with various K values and ensemble them to see what is truly best.